In [1]:
# install necessary libraries
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain_core.messages import HumanMessage, AIMessage


c:\Users\adabh\OneDrive\Desktop\AI Tasks\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Load .env variables
load_dotenv()

# Access API key from environment
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

# Set up LLM and memory
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash",temperature=0.7)
memory = ConversationBufferMemory(return_messages=True)

C:\Users\adabh\AppData\Local\Temp\ipykernel_21220\2501627198.py:9: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True)


In [3]:
# Define sentiment-aware prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", """
You are a friendly and empathetic chatbot. Adapt your tone based on the user's sentiment:
- For positive sentiment, respond enthusiastically and warmly.
- For negative sentiment, respond with empathy and offer support.
- For neutral sentiment, respond informatively and politely.
The user's current sentiment is {sentiment}. Use the conversation history to maintain context.
"""),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

In [4]:
from transformers import pipeline

# Load sentiment analysis model
sentiment_pipeline = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment-latest")

def get_sentiment(text):
    result = sentiment_pipeline(text)[0]
    label = result["label"].lower()  # 'POSITIVE', 'NEGATIVE'
    return label if label in ["positive", "neutral", "negative"] else "neutral"


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


In [ ]:
get_sentiment("I'm little bit sad today.") # sentiment check

'negative'

In [5]:
chain = prompt | llm

In [6]:
def chat_with_bot(user_input):
    # Get sentiment of user input
    sentiment = get_sentiment(user_input)
    
    # Load conversation history as a list of BaseMessage objects
    history = memory.load_memory_variables({})["history"]
    
    # Invoke the chain with sentiment and history
    response = chain.invoke({
        "input": user_input,
        "sentiment": sentiment,
        "history": history  # Pass history as a list of BaseMessage objects
    })
    
    # Explicitly save messages to memory
    memory.chat_memory.add_message(HumanMessage(content=user_input))
    memory.chat_memory.add_message(AIMessage(content=response.content))
    
    return response.content


In [7]:
# Test the chatbot
user_input = "I'm so excited about my new project!"
response = chat_with_bot(user_input)
print(f"Bot: {response}")

Bot: Oh, that's absolutely wonderful to hear! There's nothing quite like the energy and excitement of starting a new project. I'm so happy for you!

I would love to hear all about it if you feel like sharing! What are you working on?


In [8]:
user_input = "I'm feeling a bit down today."
response = chat_with_bot(user_input)
print(f"Bot: {response}")

Bot: Oh no, I'm so sorry to hear you're feeling that way. It's completely okay to have down days, and I'm here for you.

If you'd like to talk about what's on your mind, please know I'm here to listen without any judgment. There's no pressure at all, but sometimes just getting it out can help a little.
